# Chapter 13: Concurrency 🚀

- Threads
- Multiprocessing
- Futures 
- AsyncIO

**By Will Norris**

## Sequential Programming: 
- Sequential programming model is intuitive and natural
    - Do things **one step at a time** (The way humans think)
- In programming languages: 
    - Each of these real-world actions is an abstraction for a sequence of finer-grained actions
    - Flow: ```Open the cupboard, select a tea, check water level in kettle, if low: add more, boil water, pour water in cup, wait for tea...``` 
- **But**, what we do while the water is boiling is up to us
    - Do we simply wait? 
    - Or, do we do other tasks such as starting our toast or fetching the newspaper (asynchronous tasks)
        - The whole time aware that we are waiting for our water to boil!
- Tea kettle makers know people tend to operate asynchronously, so they add a warning for when your tea is done, to remind you to come back to the task at hand. 
    - Finding the right balance of sequentiality and asynchrony is a characteristic of efficient people, **the same is true of efficient programs**

### Why Concurrency? 
- At some point it is not cost efficient to buy a faster machine (**scaling vertically**)
- Instead of scaling computation up, we can go out (**scaling horizontally**)
    - Allows us to use cheap hardware, and accomplish pieces of computation across a set of threads/processors/nodes 
- In modern computing, we can divide the problem entirely across nodes (processors) 
    - In legacy computing, we could take advantage of "switching", which means rapidly swapping between threads on a single process to accompish multiple things "at once" (time sharing systems) 
    

## Multiple Processes: 
__Motivating Factors:__
- **Resource Utilizaton:** 
    - Programs are always waiting for external operations (File I/O), and can't do anything while they wait. Let's use that time!
- **Fairness:**
    - Multiple users and programs may have equal claim on the machine's resources. We want to let them share "slices" of time rather than give one before the other 
- **Convenience:**
    - It is easier to write several programs that each perform a single task and have them coordinate with each other when needed than to write one big program. 

## Threads: 
- Threads allow multiple streams of program control flow to coexist within a process. 
- They share process-wide resources (memory, file handles) 
    - But, each thread has its own program counter, stack, and local variables 
- Threads provide a natural decomposition for exploiting hardware parallelism when we have multiple processors
    -  multiple threads within the same program can be scheduled simultaneously on multi CPU's
- Most modern OS's treat threads as **lightweight processses** and use them (not processes) as the basic unit of scheduling

![](https://imgur.com/5mte34P.png)

In [1]:
from threading import Thread

class InputReader(Thread):
    def run(self):
        self.line_of_text = input()

In [2]:
print("Enter any text and press enter: ")
thread = InputReader()
thread.start()

count = result = 1
while thread.is_alive():
    result = count * count 
    count += 1

print("calculated squares up to {0} * {0} = {1}".format(
    count, result))
print("while you typed '{}'".format(thread.line_of_text))

Enter any text and press enter: 
wi
calculated squares up to 1712925 * 1712925 = 2934108629776
while you typed 'wi'


In [7]:
import json 
from urllib.request import urlopen 
import time 
import requests
import pyowm

CITIES = ['Edmonton', 'Victoria', 'Winnipeg', 'Fredericton',
          'Halifax', 'Toronto', 'Charlottetown',
          'Quebec', 'Regina']

class TempGetter(Thread):
    def __init__(self, city):
        super().__init__()
        self.city = city
        self.owm = pyowm.OWM('be06b12aa45e1ca05a8f972f81376c6d')
    def run(self):
        city = self.owm.weather_at_place(self.city)
        weather = city.get_weather()
        self.temperature = weather.get_temperature('fahrenheit')['temp']
        
threads = [TempGetter(c) for c in CITIES]
start = time.time()
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
for thread in threads:
    print("it is {0.temperature:.0f}°C in {0.city}".format(thread))
print(
   "Got {} temps in {} seconds".format(
   len(threads), time.time() - start))


it is 40°C in Edmonton
it is 79°C in Victoria
it is 31°C in Winnipeg
it is 54°C in Fredericton
it is 54°C in Halifax
it is 56°C in Toronto
it is 44°C in Charlottetown
it is 44°C in Quebec
it is 39°C in Regina
Got 9 temps in 0.19599175453186035 seconds


### What's actually happening here? 
- 10 threads are started
    - Remember to call ```super``` to ensure we instantiate an actual ```Thread``` object. 
    - We construct 10 thread objects from within the main thread, then run them later. 
        - Data constructed in one thread is accessible from other running threads
- Each thread is joined with eachother 
    - Joining threads tells each one to "wait for the thread to complete before doing anything" 
    - This means the second for loop won't end until all 10 threads have finished 
- **In threads, all state is shared by default**
    
    

### Running on one thread instead, much slower:

In [8]:
threads = [TempGetter(c) for c in CITIES]
start = time.time()
for thread in threads:
    thread.run()
for thread in threads:
    print("it is {0.temperature:.0f}°C in {0.city}".format(thread))
print(
   "Got {} temps in {} seconds".format(
   len(threads), time.time() - start))

it is 40°C in Edmonton
it is 79°C in Victoria
it is 32°C in Winnipeg
it is 54°C in Fredericton
it is 54°C in Halifax
it is 58°C in Toronto
it is 43°C in Charlottetown
it is 43°C in Quebec
it is 40°C in Regina
Got 9 temps in 1.6550121307373047 seconds


## Threads sound great! What's the catch?? 
- Python programmers avoid threading for several reasons: 
    - Better alternative methods to concurrent programming in Python
    - Shared Memory
    - Global Interpreter Lock
    - Thread Overhead

**Shared Memory:**
- Shared memory is both a major advantage and disadvantage of threading
    - It is convenient to have access to all variables in memory from any thread 
    - However, this can cause horrible inconsistencies in the program state 
        - It is easy to allow one thread to change a value that another thread expected, which can cause unknown errors. 
- We can "synchronize" thread's access to variables, however this can get complex and improper synchronization can be hard to find.

**The Global Interpreter Lock:**
- To efficiently manage memory , garbage collection, and calls to machine code in libraries, Python uses the Global Interpreter Lock (GIL)
- It is impossible to turn off and it makes it impossible to properly use threads for parralell processing in python
    - The GIL will prevent any two thread's from doing work at the exact same time, even if they have work to do. ("doing work" == using CPU) 
    - The GIL is released as soon as the thread starts to wait for anything 
    
**Why do we still have the GIL?**
- It makes the reference implementation much easier to maintain (language structure)
- It makes single core python faster

**Thread Overhead:**
- Each thread takes up some memory (both in python and the OS kernel) to keep track of the thread state 
- Switching (jumping between threads) uses some CPU time
    - This can be improved with 